In [19]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [20]:
!pip install elasticsearch

In [21]:
from elasticsearch import Elasticsearch

In [22]:
es_client = Elasticsearch('http://localhost:9200') 

In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [25]:
from tqdm.auto import tqdm

In [26]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:22<00:00, 42.61it/s]


In [27]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
query = 'I just disovered the course. Can I still join it?'
rag(query)

' Based on the provided context, you can join the course even if you discover it after its official start date. As a self-registration student (if applicable), there are no strict registration deadlines mentioned for late registrants in this specific FAQ database entry; however, they advise not to leave everything until the last minute due to potential final project submission deadlines that likely still apply regardless of your discovery timing.\n\nMoreover, if you choose a self-paced mode (as implied from other contexts), there is ongoing support available via Slack and FAQ documents for common questions, which are highly recommended before starting or joining the course to ensure preparedness as some prerequisites require setup prior to commencement.\n\nPlease note that this advice only reflects what was explicitly mentioned in your provided context; actual policies may vary based on other institutional guidelines not covered herein and should be reviewed directly from official sour

In [31]:
print(_)

 Based on the provided context, you can join the course even if you discover it after its official start date. As a self-registration student (if applicable), there are no strict registration deadlines mentioned for late registrants in this specific FAQ database entry; however, they advise not to leave everything until the last minute due to potential final project submission deadlines that likely still apply regardless of your discovery timing.

Moreover, if you choose a self-paced mode (as implied from other contexts), there is ongoing support available via Slack and FAQ documents for common questions, which are highly recommended before starting or joining the course to ensure preparedness as some prerequisites require setup prior to commencement.

Please note that this advice only reflects what was explicitly mentioned in your provided context; actual policies may vary based on other institutional guidelines not covered herein and should be reviewed directly from official sources o